## enum.Enum

Enums were introduced into the standard library in Python 3.4 and are described in the official documentation as "a set of symbolic names (members) bound to unique, constant values."  This could be extended to say that the members generally cover the concept at hand, exhausting that space with respect to the current application.  For instance, 'N E S W' would cover the cardinal directions for two-dimensional mapping.  

Another way of expressing this is to treat the members as categories or types from which objects in a system take on a distinct value.  In a relation database system, enums can simplify the data model while enforcing data integrity; no extraneous tables to manage such simple data values.  Of course, if one needs to change the enum members after the fact, then difficulties arise.  So it is best to be very sure that the enum is comprehensive and enduring. It is more flexible in Python.

The enum.Enum module has a variety of ways to assign a value to each name.  But it is sometimes the case that the application is never concerned with the value, just with the distinctness of members.  Uniqueness of the member is enforced via uniquess of the name.  Uniqueness of the values is optional but can be imposed with the @unique decorator.   


In [1]:
import enum

@enum.unique
class CardinalDirection(enum.Enum):
    NORTH = 1
    EAST  = 2
    SOUTH = 3
    WEST  = 4

print('Size of the enum: {}'.format(len(CardinalDirection)))
print('Direction is: '.format(CardinalDirection.NORTH))


Size of the enum: 4
Direction is: 


The same Enum can be created via the functional API:

In [2]:
cdirs = enum.Enum('CardinalDirection', 'NORTH EAST SOUTH WEST')
print(list(cdirs)[0])

CardinalDirection.NORTH


One subtlety is that each of the values must be equivalent to *True*, which is why the default values use an integer sequence starting with 1 rather than 0. In the case of a binary Enum, such as for RIGHT/LEFT, there is no parallel with True/False in the sense of 'not RIGHT == LEFT'.  In practice this makes little or no difference, though in pattern matching across possibilities, one might leave room for a possible invalid value or 'None'. 

One easy way to add some display flexibility is to override the dunder str display method.


In [3]:
@enum.unique
class Suit(enum.Enum):
    SPADES   = 1
    HEARTS   = 2
    DIAMONDS = 3
    CLUBS    = 4

    def __str__(self):
        return '♠♥♦♣'[self.value - 1]
    
print('{0!r} name: {1!r} symbol: {0!s}'.format(Suit.SPADES, Suit.SPADES.name))    
    

<Suit.SPADES: 1> name: 'SPADES' symbol: ♠


An elaborate example from the Python documentation is for the (eight) Planets where the value is a tuple with measurements for mass and radius.  An __init__ method is required to coordinate the fields and a @property method is defined to calculate the specific gravity.  

https://docs.python.org/3/library/enum.html

As a take-off on this, there is a StackOverflow answer by "Zero Piraeus" that uses #collections.namedtuple# as a mixin with Enum which creates slightly cleaner code with no __init__ method, despite the external declaration of the namedtuple.

https://stackoverflow.com/questions/26691784/can-named-arguments-be-used-with-python-enums


In [4]:
from collections import namedtuple
from enum import Enum

Body = namedtuple("Body", ["mass", "radius"])

class Planet(Body, Enum):

    MERCURY = Body(mass=3.303e+23, radius=2.4397e6)
    VENUS   = Body(mass=4.869e+24, radius=6.0518e6)
    EARTH   = Body(mass=5.976e+24, radius=3.3972e6)
    # ... etc.



In both cases, the function of the class as an Enum moves closer to being a Set of fleshed out objects while retaining the limited number of elements.  Viewing these from the relational perspective, it would be seen as a lookup table.

A different take on expanding the enum is to use the Cartesian product of two Enums to create a new Enum for a 2-dimensional space.  Unlike the Planets example, the following *playing card* example avoids multiple fields in favor of simply structured strings for the values.  Unfortunately for playing cards, there is no good set of Python literals to represent them.  Using the unicode symbols or numbers as the first character for the *names* isn't valid (though it is currently possible to create the enum this way and print out as a list, they can't be individually referenced in code).  But using the symbols for display #values# can is useful here.  Passing a list of (name, value) tuples works in the functional API and the list comprehension here makes it succinct.    

In [5]:
class Suit(enum.Enum):
    SPADES   = '♠'
    HEARTS   = '♥'
    DIAMONDS = '♦'
    CLUBS    = '♣'

class Rank(enum.Enum):
    A = 'A'
    K = 'K'
    Q = 'Q'
    J = 'J'
    T = '10'
    _9 = '9'
    # ... etc.

cards = enum.Enum('Cards', [ (r.name + s.name[0], (r, s)) for r in Rank for s in Suit ])
cards.__str__ = lambda self: self.value[0].value + self.value[1].value # normally the name is the display

cardlist = list(cards)
print('First 3 cards: {!s} {!s} {!s}'.format(cardlist[0], cardlist[1], cardlist[2]))
print('Royal Flush: {!s}'.format( [str(c) for c in cards if c.name[-1] == 'S'][0:5]))


First 3 cards: A♠ A♥ A♦
Royal Flush: ['A♠', 'K♠', 'Q♠', 'J♠', '10♠']


But if Poker is to be the context, then the enumeration of cards will need to be ordered and we will still need to have some display options.  So rather than use the *Functional API*, the following rewrite of the three *Enum*s uses tuples as the Enum values with an integer to provide the ordering values, a short display string, and for the cards the rank and suit.  For the cards, a custom Metaclass is used to define the Enum members which then enable the class definition format where the ordering interface is added.

In [6]:
class Suit(enum.Enum):
    SPADES   = (4, '♠')
    HEARTS   = (3, '♥')
    DIAMONDS = (2, '♦')
    CLUBS    = (1, '♣')
    

class Rank(enum.Enum):
    ACE   = (14, 'A')
    KING  = (13, 'K')
    QUEEN = (12, 'Q')
    JACK  = (11, 'J')
    TEN   = (10, '10')
    NINE  = (9,  '9')
    # etc.
    
class PlayingCardMeta(enum.EnumMeta):
    def __new__(metacls, cls, bases, classdict):
        for r in Rank:
            for s in Suit:                
                classdict[r.name + '_OF_' + s.name] = \
                    ((r.value[0] - 2) * 4 + s.value[0], r, s, r.value[1] + s.value[1])
            
        return super(PlayingCardMeta, metacls).__new__(metacls, cls, bases, classdict)

    
class PlayingCard(enum.Enum, metaclass=PlayingCardMeta):

    @property
    def rank(self):
        return self.value[1].name
    
    @property    
    def suit(self):
        return self.value[2].name
            
    def __repr__(self): 
        return self.name

    def __str__(self): 
        return self.value[3]

    def __ge__(self, other):
        if self.__class__ is other.__class__:
            return self.value[0] >= other.value[0]
        return NotImplemented
    def __gt__(self, other):
        if self.__class__ is other.__class__:
            return self.value[0] > other.value[0]
        return NotImplemented
    def __le__(self, other):
        if self.__class__ is other.__class__:
            return self.value[0] <= other.value[0]
        return NotImplemented
    def __lt__(self, other):
        if self.__class__ is other.__class__:
            return self.value[0] < other.value[0]
        return NotImplemented

        
print(list(PlayingCard)[0:5])
print('{} has suit {} and rank {}'.format(PlayingCard.QUEEN_OF_DIAMONDS, 
                              PlayingCard.QUEEN_OF_DIAMONDS.suit,
                              PlayingCard.QUEEN_OF_DIAMONDS.rank))       
print('{} beats {} : {}'.format(PlayingCard.ACE_OF_SPADES, PlayingCard.KING_OF_CLUBS,
                                PlayingCard.ACE_OF_SPADES >= PlayingCard.KING_OF_CLUBS))        


[ACE_OF_SPADES, ACE_OF_HEARTS, ACE_OF_DIAMONDS, ACE_OF_CLUBS, KING_OF_SPADES]
Q♦ has suit DIAMONDS and rank QUEEN
A♠ beats K♣ : True
